In [17]:
import abc
import random
import copy

In [18]:
class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement
    legal_moves, make_move, utility, and terminal_test. You may
    override display and successors or you can inherit their default
    methods. You will also need to set the .initial attribute to the
    initial state; this can be done in the constructor."""
    @abc.abstractmethod
    def legal_moves(self, state):
        "Return a list of the allowable moves at this point."
    @abc.abstractmethod
    def make_move(self, move, state):
        "Return the state that results from making a move from a state."
    @abc.abstractmethod
    def utility(self, state, player):
        "Return the value of this final state to player."
    @abc.abstractmethod
    def terminal_test(self, state):
        "Return True if this is a final state for the game."
        return not self.legal_moves(state)
    def to_move(self, state):
        "Return the player whose move it is in this state."
        return state.to_move
    def display(self, state):
        "Print or otherwise display the state."
        print state
    def successors(self, state):
        "Return a list of legal (move, state) pairs."
        return [(move, self.make_move(move, state))
                for move in self.legal_moves(state)]
    def __repr__(self):
        return '<%s>' % self.__class__.__name__

In [19]:
def argmin(seq, fn):
    """Return an element with lowest fn(seq[i]) score; tie goes to first one.
    >>> argmin(['one', 'to', 'three'], len)
    'to'
    """
    best = seq[0]; best_score = fn(best)
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score
    return best
def argmax(seq, fn):
    """Return an element with highest fn(seq[i]) score; tie goes to first one.
    >>> argmax(['one', 'to', 'three'], len)
    'three'
    """
    return argmin(seq, lambda x: -fn(x))

In [20]:
def alphabeta_search(state, game, d=float('inf'), cutoff_test=None, eval_fn=None):
        """Search game to determine best action; use alpha-beta pruning.
        This version cuts off search and uses an evaluation function."""
        player = game.to_move(state)
        def max_value(state, alpha, beta, depth):
            if cutoff_test != None:
                return eval_fn(state, player)
            v = eval_fn(state, player)
            for (a, s) in game.successors(state):
                v = max(v, min_value(s, alpha, beta, depth+1))
                if v >= beta:
                    return v
                alpha = max(alpha, v)
            return v
    
        def min_value(state, alpha, beta, depth):
            if cutoff_test != None:
                return eval_fn(state, player)
            v = eval_fn(state, player)
            for (a, s) in game.successors(state):
                v = min(v, max_value(s, alpha, beta, depth+1))
                if v <= alpha:
                    return v
                beta = min(beta, v)
            return v
    
        # Body of alphabeta_search starts here:
        # The default test cuts off at depth d or at a terminal state
        cutoff_test = (cutoff_test or
                       (lambda state,depth: depth>d or game.terminal_test(state)))
        eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
        action, state = argmax(game.successors(state),
                               lambda ((a, s)): min_value(s, -float('inf'), float('inf'), 0))
        print(action)
        return action

In [21]:
def isnumber(x):
    "Is x a number? We say it is if it has a __int__ method."
    return hasattr(x, '__int__')
    
def num_or_str(x):
    """The argument is a string; convert to a number if possible, or strip it.
    >>> num_or_str('42')
    42
    >>> num_or_str(' 42x ')
    '42x'
    """
    if isnumber(x): return x
    elif(',' in x):
        x = x.replace('(', '').replace(')', '')
        splitArray = x.split(',')
        return (int(splitArray[0].replace(',', '')), int(splitArray[1].replace(',', '')))
    try:
        return int(x)
    except ValueError:
        try:
            return float(x)
        except ValueError:
                return str(x).strip()

In [22]:
def query_player(game, state):
        "Make a move by querying standard input."
        return num_or_str(raw_input('Your move? '))
    
def random_player(game, state):
        "A player that chooses a legal move at random."
        return random.choice(game.legal_moves(state))
    
def alphabeta_player(game, state):
    return alphabeta_search(state, game)

def smart_player_last_stone(game, state):
    return alphabeta_search(state, game, float('inf'),None, eval_fn_last_stone)
    
def smart_player_three_last_stone(game, state):
    return alphabeta_search(state, game, float('inf'),None, eval_fn_three_heap_last_stone)
    
def smart_player_open_field_tic_tac_toe( game, state):
    return alphabeta_search(state, game, float('inf'),None, eval_fn_three_heap_last_stone)

In [23]:
'''Function based on Nim Sum algorithm'''
def eval_fn_last_stone(state, player):
    binaryRepresentation = list('{0:0b}'.format(state.heap))
        
    count = 1
    if(binaryRepresentation ==['0']):
        return 2
    while count < len(binaryRepresentation):
            
        if(binaryRepresentation[count] != '0'):
            return 0
            
        count+= 1
        
    return 1
    
'''Function based on Nim Sum algorithm'''
def eval_fn_three_heap_last_stone(state, player):
    binaryRepresentationFirst = list('{0:0b}'.format(state.heaps[0]))
    binaryRepresentationSecond = list('{0:0b}'.format(state.heaps[1]))
    binaryRepresentationThird = list('{0:0b}'.format(state.heaps[2]))
        
    count = 1
        
    moveableValue = False
    if(binaryRepresentationFirst == binaryRepresentationSecond):
        if(binaryRepresentationThird ==['0']):
            return 2
        while count < len(binaryRepresentationThird):
            
            if(binaryRepresentationThird[count] != '0'):
                return 0
                
            count+= 1
    elif(binaryRepresentationSecond == binaryRepresentationThird):
        if(binaryRepresentationFirst ==['0']):
            return 2
        while count < len(binaryRepresentationFirst):
            
            if(binaryRepresentationFirst[count] != '0'):
                return 0
                
            count+= 1
    elif(binaryRepresentationFirst == binaryRepresentationThird):
        if(binaryRepresentationSecond ==['0']):
            return 2
        while count < len(binaryRepresentationSecond):
            
            if(binaryRepresentationSecond[count] != '0'):
                return 0
                
            count+= 1
    else:
            
        binaryRepresentationFirst.reverse()
        binaryRepresentationSecond.reverse()
        binaryRepresentationThird.reverse()
            
        count = 0
        while(count < max(len(binaryRepresentationFirst),len(binaryRepresentationSecond),len(binaryRepresentationThird)) and not moveableValue):
            aux = 0;
            if(len(binaryRepresentationFirst) > count):
                aux += float(binaryRepresentationFirst[count])
            if(len(binaryRepresentationSecond) > count):
                aux += float(binaryRepresentationSecond[count])
            if(len(binaryRepresentationThird) > count):
                aux += float(binaryRepresentationThird[count])
                
            aux = int(aux%2)
                
            if(aux != 0):
                return 0
                
            count += 1
        
    return 1

In [24]:
def play_game(game, *players):
        "Play an n-person, move-alternating game."
        state = game.initial
        print(state)
        while True:
            for player in players:
                print str(player)
                move = player(game, state)
                state = game.make_move(move, state)
                print(state)
                if game.terminal_test(state):
                    return game.utility(state, 0)

In [25]:
class Struct:
    """Create an instance with argument=value slots.
    This is for making a lightweight object whose class doesn't matter."""
    def __init__(self, **entries):
        self.__dict__.update(entries)

    def __cmp__(self, other):
        if isinstance(other, Struct):
            return cmp(self.__dict__, other.__dict__)
        else:
            return cmp(self.__dict__, other)

    def __repr__(self):
        args = ['%s=%s' % (k, repr(v)) for (k, v) in vars(self).items()]
        return 'Struct(%s)' % ', '.join(args)

In [26]:
class LastStone(Game):
    def __init__(self, stones):
        self.initial = Struct(to_move=0, heap = stones)

    def legal_moves(self, state):
        "Return a list of the allowable moves at this point."
        return range(1, min(3, state.heap) + 1)

    def make_move(self, move, state):
        "Return the state that results from making a move from a state."
        return Struct(to_move = 1 - state.to_move,
                      heap = state.heap - move)
        
    def utility(self, state, player):
        "Return the value of this final state to player."
        if state.to_move == player:
            return -1
        else:
            return 1

    def terminal_test(self, state):
        "Return True if this is a final state for the game."
        return not self.legal_moves(state)

    def to_move(self, state):
        "Return the player whose move it is in this state."
        return state.to_move

    def display(self, state):
        "Print or otherwise display the state."
        print state

    def successors(self, state):
        "Return a list of legal (move, state) pairs."
        return [(move, self.make_move(move, state))
                for move in self.legal_moves(state)]

In [27]:
class ThreeHeapLastStone(Game):
    def __init__(self, heaps):
        self.initial = Struct(to_move=0, heaps = heaps)

    def legal_moves(self, state):
        "Return a list of the allowable moves at this point."        
        legalMoves = []
        
        for x in range (0,3):
            if(state.heaps[x] != 0):
                legalMoves.append((x,range(1, min(3, state.heaps[x]) + 1)))
                
        return legalMoves

    def make_move(self, move, state):
        "Return the state that results from making a move from a state."
        heaps = copy.copy(state.heaps)
        heaps[move[0]] = state.heaps[move[0]] - move[1]
        return Struct(to_move = 1 - state.to_move,
                      heaps = heaps)
        
    def utility(self, state, player):
        "Return the value of this final state to player."
        if state.to_move == player:
            return -1
        else:
            return 1
        
    def terminal_test(self, state):
        "Return True if this is a final state for the game."
        return not self.legal_moves(state)

    def to_move(self, state):
        "Return the player whose move it is in this state."
        return state.to_move
    
    def display(self, state):
        "Print or otherwise display the state."
        print state

    def successors(self, state):
        "Return a list of legal (move, state) pairs."
        
        succesorMoves = []
        
        for heapMoves in self.legal_moves(state):
            for move in heapMoves[1]:
                wholeMove = (heapMoves[0], move)
                succesorMoves.append((wholeMove, self.make_move(wholeMove, state)))
        return succesorMoves

In [33]:
def play():
    result = play_game(LastStone(15), smart_player_last_stone, query_player)
    if result == 1:
        print "Smart player wins"
    else:
        print "Smart player loses"
            
def playTest():
    result = play_game(LastStone(10), smart_player_last_stone, alphabeta_player)
    if result == 1:
        print "Player wins"
    else:
        print "CPU wins"
            
def playThreeHeapsTest():
    result = play_game(ThreeHeapLastStone([10,10,10]), smart_player_three_last_stone, alphabeta_player)
    if result == 1:
        print "Smart player wins"
    else:
        print "Smart player loses"
    
def playThreeHeaps():
    result = play_game(ThreeHeapLastStone([10,10,10]), query_player, smart_player_three_last_stone )
    if result == 1:
        print "CPU wins"
    else:
        print "Player wins"
            
            


In [32]:
playThreeHeaps()

Struct(heaps=[10, 10, 10], to_move=0)
<function query_player at 0x7feac3471758>


KeyboardInterrupt: 

In [34]:
play()

Struct(to_move=0, heap=15)
<function smart_player_last_stone at 0x7feac3462de8>
1
Struct(to_move=1, heap=14)
<function query_player at 0x7feac3471758>
Your move? 1
Struct(to_move=0, heap=13)
<function smart_player_last_stone at 0x7feac3462de8>
1
Struct(to_move=1, heap=12)
<function query_player at 0x7feac3471758>
Your move? 3
Struct(to_move=0, heap=9)
<function smart_player_last_stone at 0x7feac3462de8>
1
Struct(to_move=1, heap=8)
<function query_player at 0x7feac3471758>
Your move? 2
Struct(to_move=0, heap=6)
<function smart_player_last_stone at 0x7feac3462de8>
2
Struct(to_move=1, heap=4)
<function query_player at 0x7feac3471758>
Your move? 1
Struct(to_move=0, heap=3)
<function smart_player_last_stone at 0x7feac3462de8>
3
Struct(to_move=1, heap=0)
Smart player wins


Struct(heaps=[10, 10, 10], to_move=0)
<function smart_player_three_last_stone at 0x7f3ce00c4398>
(0, 2)
Struct(heaps=[8, 10, 10], to_move=1)
<function query_player at 0x7f3ce00a9ed8>
Your move? (0,2)
Struct(heaps=[6, 10, 10], to_move=0)
<function smart_player_three_last_stone at 0x7f3ce00c4398>
(0, 2)
Struct(heaps=[4, 10, 10], to_move=1)
<function query_player at 0x7f3ce00a9ed8>
Your move? (0,3)
Struct(heaps=[1, 10, 10], to_move=0)
<function smart_player_three_last_stone at 0x7f3ce00c4398>
(0, 1)
Struct(heaps=[0, 10, 10], to_move=1)
<function query_player at 0x7f3ce00a9ed8>
Your move? (1,3)
Struct(heaps=[0, 7, 10], to_move=0)
<function smart_player_three_last_stone at 0x7f3ce00c4398>
(2, 3)
Struct(heaps=[0, 7, 7], to_move=1)
<function query_player at 0x7f3ce00a9ed8>
Your move? (0,1)
Struct(heaps=[-1, 7, 7], to_move=0)
<function smart_player_three_last_stone at 0x7f3ce00c4398>


ValueError: could not convert string to float: -